#  Indicators of World Development and COVID19 Impacts

##  The aim of the Project is to analysis to answer interesting questions about the world development indicators and COVID19 Impacts dataset that was provided by the World Bank and

###  Loading libraries

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from scipy import stats
import numpy as np

from datetime import datetime
from dateutil.parser import parse
from pandas.tseries.offsets import Hour, Minute

### Initial Data Analysis EDA

#### Environment, Social And Governance Data - .csv file to Dataframe

In [19]:
df_Country = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/ESG_csv/ESGCountry.csv')
df_ESGData = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/ESG_csv/ESGData.csv')
df_ESGSeriesData = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/ESG_csv/ESGCountry-Series.csv')

#### Health Nutrition And Population Data - .csv file to Dataframe

In [20]:
df_HNPQData = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/hnpq_csv/HNPQData.csv')
df_HNPQSeriesData = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/hnpq_csv/HNPQSeries.csv')

#### COVID19 Impact on Daily Life - .csv file to Dataframe

In [21]:
df_COVIDData = pd.read_csv('C:/Users/ragun/Documents/GitHub/DSC540/Weeks 5 & 6/Dataset/COVID19 Impact/Global_Mobility_Report.csv')

C:\Users\ragun\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
#df_Country.describe
#df_Country.head(5)

#df_ESGData.describe
#df_ESGData.head(5)

#df_HNPQData.describe
#df_HNPQData.head(5)

## Data Transfer and Clean Up - Country Dataset

In [23]:
# Renaming the Columns
mapping = {'Country Code':'country_code', 'Short Name':'Country', 'Table Name':'Country_Name', 'Long Name':'Country_FullName'}
df_Country.rename(columns=mapping, inplace=True)

# Dropping few unwanted columns
df_Country.drop(columns='Special Notes', inplace=True)
df_Country.drop(columns='System of trade', inplace=True)

# Removing the data with NA data
df_Country.Country.dropna()

#Replacing Nan values wiht No Data
df_Country['Latest population census'].fillna('No Data')

# Verifying if any duplicates, null available in the dataset
print("Country Name is duplictaed - {}".format(any(df_Country.Country.duplicated())))
print("Country Name contains NaN - %r " % df_Country.Country.isnull().values.any())


Country Name is duplictaed - False
Country Name contains NaN - False 


### Fuzzy Matching on the County and Country Full Name

In [24]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [25]:
str2Match = df_Country['Country'].fillna('######').all()
strOptions = df_Country['Country_FullName'].fillna('######').all()

name_match,ratio_match=checker(str2Match,strOptions)

In [26]:
fuzz.ratio(str2Match,strOptions)

57

### identifying the outliers

In [27]:
Q1 = df_Country.quantile(0.25)
Q3 = df_Country.quantile(0.75)
IQR = Q3 - Q1
print(IQR)
#print(df_Country < (Q1 - 1.5 * IQR)) |(df_Country > (Q3 + 1.5 * IQR))

National accounts reference year     5.0
PPP survey year                      NaN
Latest industrial data              11.0
Latest trade data                    0.0
Unnamed: 30                          NaN
dtype: float64


## Data Transfer and Clean Up - Environment, Social And Governance Data

In [28]:
# Renaming the Columns
mapping = {'Country Code':'country_code', 'Indicator Code':'Indicator', 'Country Name':'Country_Name', 
           '2019':'PreviousYear', '2020':'CurrentYear'}
df_ESGData.rename(columns=mapping, inplace=True) 

# Dropping few unwanted columns
df_ESGData.drop(columns =['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969']) 

# Removing the data with NA data
df_ESGData.Country_Name.dropna()

#Replacing Nan values wiht No Data
df_ESGData['CurrentYear'].fillna('No Data')
df_ESGData['PreviousYear'].fillna('No Data')

# Verifying if any duplicates, null available in the dataset
print("Country Name is duplictaed - {}".format(any(df_ESGData.Indicator.duplicated())))
print("Country Name contains NaN - %r " % df_ESGData.isnull().values.any())
 



Country Name is duplictaed - True
Country Name contains NaN - True 


In [29]:
df_ESGData_unpivoted = df_ESGData.melt(['Country_Name','Indicator'], var_name='Year', value_name='Values')
df_ESGData_unpivoted

,Country_Name,Indicator,Year,Values
0,Arab World,EG.CFT.ACCS.ZS,country_code,ARB
1,Arab World,EG.ELC.ACCS.ZS,country_code,ARB
2,Arab World,NY.ADJ.DRES.GN.ZS,country_code,ARB
3,Arab World,NY.ADJ.DFOR.GN.ZS,country_code,ARB
4,Arab World,AG.LND.AGRI.ZS,country_code,ARB
...,...,...,...,...
1040840,Zimbabwe,IC.LGL.CRED.XQ,Unnamed: 66,NaN
1040841,Zimbabwe,ER.PTD.TOTL.ZS,Unnamed: 66,NaN
1040842,Zimbabwe,SL.UEM.TOTL.ZS,Unnamed: 66,NaN
1040843,Zimbabwe,SP.UWT.TFRT,Unnamed: 66,NaN


In [30]:
df_ESGData.Indicator.value_counts()


EN.ATM.NOXE.PC       239
SE.ENR.PRSC.FM.ZS    239
SH.STA.SMSS.ZS       239
SP.UWT.TFRT          239
VA.EST               239
                    ... 
EG.EGY.PRIM.PP.KD    239
SE.ADT.LITR.ZS       239
SN.ITK.DEFC.ZS       239
EG.USE.COMM.FO.ZS    239
SL.UEM.TOTL.ZS       239
Name: Indicator, Length: 67, dtype: int64

## Data Transfer and Clean Up - Health Nutrition And Population

In [31]:
# Renaming the Columns
mapping = {'Country Code':'country_code', 'Indicator Code':'Indicator', 'Country Name':'Country_Name', 
           '2019':'PreviousYear', '2020':'CurrentYear'}
df_HNPQData.rename(columns=mapping, inplace=True) 

# Dropping few unwanted columns
df_HNPQData.drop(columns =['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999']) 

# Removing the data with NA data
df_HNPQData.Country_Name.dropna()

#Replacing Nan values wiht No Data
df_HNPQData['CurrentYear'].fillna('No Data')
df_HNPQData['PreviousYear'].fillna('No Data')

# Verifying if any duplicates, null available in the dataset
print("Country Name is duplictaed - {}".format(any(df_HNPQData.Indicator.duplicated())))
print("Country Name contains NaN - %r " % df_HNPQData.isnull().values.any())

Country Name is duplictaed - True
Country Name contains NaN - True 


In [32]:
df_HNPQData.Indicator.value_counts()

SH.STA.MLN3.Q3.ZS    217
SH.STA.STN3.Q1.ZS    217
SH.FPL.ACPT.Q4.ZS    217
SP.UWT.TFRT.Q5.ZS    217
SP.DYN.WFRT.Q2       217
                    ... 
SH.FPL.MBRI.Q5       217
SH.STA.ANCP.Q4.ZS    217
SP.MTR.1519.Q1.ZS    217
SH.STA.ARIC.Q1.ZS    217
SH.STA.STN3.Q5.ZS    217
Name: Indicator, Length: 420, dtype: int64

## Data Transfer and Clean Up - COVID19

In [33]:
# Renaming the Columns
mapping = {'country_region':'Country_Name', 
           }
df_COVIDData.rename(columns=mapping, inplace=True) 

# Dropping few unwanted columns
df_COVIDData.drop(columns =['iso_3166_2_code','census_fips_code']) 

# Removing the data with NA data
df_COVIDData.Country_Name.dropna()


# Verifying if any duplicates, null available in the dataset
print("Country Name is duplictaed - {}".format(any(df_COVIDData.Country_Name.duplicated())))
print("Country Name contains NaN - %r " % df_COVIDData.isnull().values.any())

Country Name is duplictaed - True
Country Name contains NaN - True 


In [34]:
df_COVIDData

,country_region_code,Country_Name,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4039490,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-21,NaN,NaN,NaN,NaN,-36.0,NaN
4039491,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-22,NaN,NaN,NaN,NaN,-28.0,NaN
4039492,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-23,NaN,NaN,NaN,NaN,-34.0,NaN
4039493,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-25,NaN,NaN,NaN,NaN,-39.0,NaN


In [35]:
df_COVIDData.isnull().sum()

country_region_code                                      2572
Country_Name                                                0
sub_region_1                                            69043
sub_region_2                                           674382
metro_area                                            4017075
iso_3166_2_code                                       3315028
census_fips_code                                      3177555
date                                                        0
retail_and_recreation_percent_change_from_baseline    1496394
grocery_and_pharmacy_percent_change_from_baseline     1584485
parks_percent_change_from_baseline                    2103752
transit_stations_percent_change_from_baseline         1996979
workplaces_percent_change_from_baseline                191363
residential_percent_change_from_baseline              1698329
dtype: int64

In [40]:
df_COVIDData.dropna(axis=0,inplace=True)
df_Country.dropna(axis=0,inplace=True)

In [42]:
df_COVIDData['country_region_code'].fillna('No Region', inplace=True)
df_Country['country_code'].fillna('No Country', inplace=True)

Series([], Name: country_code, dtype: object)

In [43]:
df_COVIDData.duplicated()

Series([], dtype: bool)

In [44]:
df_COVIDData['country_region_code'].replace(to_replace = np.nan, value = 0,inplace=True)

Series([], Name: country_region_code, dtype: object)

In [45]:
pd.merge(df_COVIDData, df_Country)

,country_region_code,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Unnamed: 30


In [46]:
df_COVIDData.groupby(df_COVIDData["country_region_code"]).sum()

,census_fips_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
country_region_code,,,,,,,


In [49]:
now = datetime.now()
now
now.year, now.month, now.day

(2021, 2, 11)

In [50]:
index = pd.date_range('2021-02-01', '2021-02-02')
index

DatetimeIndex(['2021-02-01', '2021-02-02'], dtype='datetime64[ns]', freq='D')

In [51]:
p = pd.Period('2021', freq='A-DEC')
p
p.asfreq('M', how='start')
p.asfreq('M', how='end')

Period('2021-12', 'M')

## References

World Development Indicators

https://datacatalog.worldbank.org/dataset/environment-social-and-governance-data

https://datacatalog.worldbank.org/dataset/health-nutrition-and-population-statistics-wealth-quintile

https://www.kaggle.com/shabou/world-developement-indicators-environment?select=WDI-Environment.csv


COVID 19 Indicators

http://pubdocs.worldbank.org/en/589511605042232186/COVID-19-Hzigh-Frequency-Monitoring-Dashboard-Data-Dictionary.xls
https://www.google.com/covid19/mobility/